In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import my_utils
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import pywt
import pywt.data
scaler = MinMaxScaler()

In [2]:
wavelet = 'haar'
level = 5
threshold = 0.7
# Perform the DWT

def soft_threshold(value, threshold):
    return np.where(value > threshold, value - threshold, np.where(value < -threshold, value + threshold, 0.0))

# Initialize lists to store train and test sets
train_sets = []
test_sets = []

# Define the sliding window size (20% of the data)
window_size = 0.2

# Loop through the files and create train-test sets
for j in range(5):  # For each fold
    # Initialize lists to store train and test sets for this fold
    fold_train_sets = []
    fold_test_sets = []
    
    for i in range(1, 9):  # For each file
        data_amp_original, data_pha = pd.read_csv(f'..\Data\DataPreprocessed\\fra{i}_w51_hamp_sf08.csv'), pd.read_csv(f'..\Data\DataPreprocessed\\fra{i}_calibratedPha.csv')
        coeffs = pywt.wavedec(data_amp_original, wavelet, level=level, axis=0)
        denoised_coeffs = [soft_threshold(coeff, threshold) for coeff in coeffs]
        data_amp = pywt.waverec(denoised_coeffs, wavelet, axis=0)
        if (data_amp_original.shape[0] % 2) == 1:
            data_amp = data_amp[:-1]
        data_amp = pd.DataFrame(scaler.fit_transform(data_amp))

        dataset = pd.DataFrame(my_utils.make_alternating(data_amp, data_pha))
        dataset = my_utils.create_tensor(dataset)
        
        # Calculate the number of data points for the test set and train set
        total_points = len(dataset)
        test_size = int(total_points * window_size)
        train_size = total_points - test_size
        
        # Calculate the starting index for the test set
        test_start = j * test_size
        
        # Calculate the ending index for the test set
        test_end = test_start + test_size
        
        # Extract test and train sets using slicing
        test_set = dataset[test_start:test_end]
        train_set = np.concatenate([dataset[:test_start], dataset[test_end:]])
        
        # Append train and test sets to the respective fold lists
        fold_train_sets.append(train_set)
        fold_test_sets.append(test_set)
    
    # Merge train sets and test sets for this fold
    train_sets.append(np.concatenate(fold_train_sets))
    test_sets.append(np.concatenate(fold_test_sets))

# Now you have 5 train-test set couples for cross-validation

In [3]:
tensorFra = np.array(train_sets)
tensorTestFra = np.array(test_sets)

In [4]:
# Initialize lists to store train and test sets
train_sets = []
test_sets = []

# Define the sliding window size (20% of the data)
window_size = 0.2

# Loop through the files and create train-test sets
for j in range(5):  # For each fold
    # Initialize lists to store train and test sets for this fold
    fold_train_sets = []
    fold_test_sets = []
    
    for i in range(1, 8):  # For each file
        data_amp_original, data_pha = pd.read_csv(f'..\Data\DataPreprocessed\laura{i}_w51_hamp_sf08.csv'), pd.read_csv(f'..\Data\DataPreprocessed\laura{i}_calibratedPha.csv')

        coeffs = pywt.wavedec(data_amp_original, wavelet, level=level, axis=0)
        denoised_coeffs = [soft_threshold(coeff, threshold) for coeff in coeffs]
        data_amp = pywt.waverec(denoised_coeffs, wavelet, axis=0)
        if (data_amp_original.shape[0] % 2) == 1:
            data_amp = data_amp[:-1]

        data_amp = pd.DataFrame(scaler.fit_transform(data_amp))
        dataset = pd.DataFrame(my_utils.make_alternating(data_amp, data_pha))
        dataset = my_utils.create_tensor(dataset)
        
        # Calculate the number of data points for the test set and train set
        total_points = len(dataset)
        test_size = int(total_points * window_size)
        train_size = total_points - test_size
        
        # Calculate the starting index for the test set
        test_start = j * test_size
        
        # Calculate the ending index for the test set
        test_end = test_start + test_size
        
        # Extract test and train sets using slicing
        test_set = dataset[test_start:test_end]
        train_set = np.concatenate([dataset[:test_start], dataset[test_end:]])
        
        # Append train and test sets to the respective fold lists
        fold_train_sets.append(train_set)
        fold_test_sets.append(test_set)
    
    for i in range(1, 6):  # For each file
        data_amp_original, data_pha = pd.read_csv(f'..\Data\DataPreprocessed\lauraDay2_{i}_w51_hamp_sf08.csv'), pd.read_csv(f'..\Data\DataPreprocessed\lauraDay2_{i}_calibratedPha.csv')

        coeffs = pywt.wavedec(data_amp_original, wavelet, level=level, axis=0)
        denoised_coeffs = [soft_threshold(coeff, threshold) for coeff in coeffs]
        data_amp = pywt.waverec(denoised_coeffs, wavelet, axis=0)
        if (data_amp_original.shape[0] % 2) == 1:
            data_amp = data_amp[:-1]

        data_amp = pd.DataFrame(scaler.fit_transform(data_amp))
        dataset = pd.DataFrame(my_utils.make_alternating(data_amp, data_pha))
        dataset = my_utils.create_tensor(dataset)
        
        # Calculate the number of data points for the test set and train set
        total_points = len(dataset)
        test_size = int(total_points * window_size)
        train_size = total_points - test_size
        
        # Calculate the starting index for the test set
        test_start = j * test_size
        
        # Calculate the ending index for the test set
        test_end = test_start + test_size
        
        # Extract test and train sets using slicing
        test_set = dataset[test_start:test_end]
        train_set = np.concatenate([dataset[:test_start], dataset[test_end:]])
        
        # Append train and test sets to the respective fold lists
        fold_train_sets.append(train_set)
        fold_test_sets.append(test_set)

    # Merge train sets and test sets for this fold
    train_sets.append(np.concatenate(fold_train_sets))
    test_sets.append(np.concatenate(fold_test_sets))

# Now you have 5 train-test set couples for cross-validation

In [5]:
tensorLaura = np.array(train_sets)
tensorTestLaura = np.array(test_sets)

In [8]:
print(tensorFra.shape, tensorTestFra.shape)
print(tensorLaura.shape, tensorTestLaura.shape)

(5, 755, 200, 104) (5, 184, 200, 104)
(5, 875, 200, 104) (5, 213, 200, 104)


In [6]:
# Initialize lists to store train and test sets
train_sets = []
test_sets = []

# Define the sliding window size (20% of the data)
window_size = 0.2

# Loop through the files and create train-test sets
for j in range(5):  # For each fold
    # Initialize lists to store train and test sets for this fold
    fold_train_sets = []
    fold_test_sets = []
    
    for i in range(1, 7):  # For each file
        data_amp_original, data_pha = pd.read_csv(f'..\Data\DataPreprocessed\gio{i}_w51_hamp_sf08.csv'), pd.read_csv(f'..\Data\DataPreprocessed\gio{i}_calibratedPha.csv')

        coeffs = pywt.wavedec(data_amp_original, wavelet, level=level, axis=0)
        denoised_coeffs = [soft_threshold(coeff, threshold) for coeff in coeffs]
        data_amp = pywt.waverec(denoised_coeffs, wavelet, axis=0)
        if (data_amp_original.shape[0] % 2) == 1:
            data_amp = data_amp[:-1]

        data_amp = pd.DataFrame(scaler.fit_transform(data_amp))
        dataset = pd.DataFrame(my_utils.make_alternating(data_amp, data_pha))
        dataset = my_utils.create_tensor(dataset)
        
        # Calculate the number of data points for the test set and train set
        total_points = len(dataset)
        test_size = int(total_points * window_size)
        train_size = total_points - test_size
        
        # Calculate the starting index for the test set
        test_start = j * test_size
        
        # Calculate the ending index for the test set
        test_end = test_start + test_size
        
        # Extract test and train sets using slicing
        test_set = dataset[test_start:test_end]
        train_set = np.concatenate([dataset[:test_start], dataset[test_end:]])
        
        # Append train and test sets to the respective fold lists
        fold_train_sets.append(train_set)
        fold_test_sets.append(test_set)
    
    # Merge train sets and test sets for this fold
    train_sets.append(np.concatenate(fold_train_sets))
    test_sets.append(np.concatenate(fold_test_sets))

# Now you have 5 train-test set couples for cross-validation

In [7]:
tensorGio = np.array(train_sets)
tensorTestGio = np.array(test_sets)

In [14]:
print(tensorFra.shape, tensorTestFra.shape)
print(tensorLaura.shape, tensorTestLaura.shape)
print(tensorGio.shape, tensorTestGio.shape)

(5, 755, 200, 104) (5, 184, 200, 104)
(5, 875, 200, 104) (5, 213, 200, 104)
(5, 723, 200, 104) (5, 179, 200, 104)


In [8]:
for i in range(5):
    #tensor1 = torch.from_numpy(tensorEmpty)
    tensor2 = torch.from_numpy(tensorGio[i])
    tensor3 = torch.from_numpy(tensorLaura[i])
    tensor4 = torch.from_numpy(tensorFra[i])
    #tensor5 = torch.from_numpy(tensorEmpty)
    # Assuming you have your data tensors tensor1 and tensor2 of shape (1135, 200, 104)
    # Add a channel dimension to the input data
    #tensor1 = tensor1.unsqueeze(1)  # Adds a channel dimension (1) at position 1
    tensor2 = tensor2.unsqueeze(1)
    tensor3 = tensor3.unsqueeze(1)
    tensor4 = tensor4.unsqueeze(1)
    #tensor5 = tensor5.unsqueeze(1)
    # Assuming you have 455 images of class 1 and 680 images of class 2
    #labels1 = torch.zeros(1103, dtype=torch.long)  # Class 1 labels as 0
    labels2 = torch.zeros(723, dtype=torch.long)   # Class 2 labels as 1
    labels3 = torch.ones(875, dtype=torch.long) # Class 3 labels as 2
    labels4 = torch.ones(755, dtype=torch.long) + 1
    #labels5 = torch.ones(895, dtype=torch.long) + 2
    # Combine Data and Labels
    data = torch.cat((tensor2, tensor3, tensor4), dim=0)
    labels = torch.cat((labels2, labels3, labels4), dim=0)

    #dataset = TensorDataset(data, labels) #dataset = TensorDataset(data, labels)
    ### dataset = torch.load('..\Data\DatasetTensor\empty_laura.pt')
    train_dataset = TensorDataset(data,labels)

    tt2 = torch.from_numpy(tensorTestGio[i])
    tt3 = torch.from_numpy(tensorTestLaura[i])
    tt4 = torch.from_numpy(tensorTestFra[i])
    #tt5 = torch.from_numpy(tensorTestEmpty)
    tt2 = tt2.unsqueeze(1)
    tt3 = tt3.unsqueeze(1)
    tt4 = tt4.unsqueeze(1)
    #tt5 = tt5.unsqueeze(1)
    ll2 = torch.zeros(179, dtype=torch.long)
    ll3 = torch.ones(213, dtype=torch.long)
    ll4 = torch.ones(184, dtype=torch.long) + 1
    #ll5 = torch.ones(208, dtype=torch.long) + 2
    dataT = torch.cat((tt2, tt3, tt4), dim=0)
    labelsT = torch.cat((ll2, ll3, ll4), dim=0)
    test_dataset = TensorDataset(dataT,labelsT)
    '''
    tt1 = torch.from_numpy(empty_test).unsqueeze(1)
    tt2 = torch.from_numpy(laura_test).unsqueeze(1)
    ll1 = torch.zeros(123, dtype=torch.long)
    ll2 = torch.ones(125, dtype=torch.long)
    dd = torch.cat((tt1, tt2), dim=0)
    ll = torch.cat((ll1, ll2), dim=0)
    test_dataset = TensorDataset(dd,ll)
    '''
    # Define the sizes for your train and test sets
    #train_size = int(0.8 * len(dataset))  # 80% for training
    #test_size = len(dataset) - train_size  # 20% for testing
    # Split the dataset into train and test
    #train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    # Create DataLoaders
    batch_size = 64  # Adjust as neededs
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    torch.save(train_dataset,f'models\\train_DWT_GioLaFra_{i}.pt')
    torch.save(test_dataset,f'models\\test_DWT_GioLaFra_{i}.pt')

5-fold cv LaFra fraGio   GioLA                      Ternary-class

86.7           88.5            94.6                 85.2
82.6           92.3            93.9                 89.2
90.5           93.7            98.5                 96
88.6           93.1            92.9                 92.5
95.5           88.4            92.1                 89.2
               Medie
88.78           91.2           94.4s                90.4
            media tot: 91.5

            Train 100 epochs 0.001, 20 epochs 0.0005

with DWT
91.2            94.2            94.4                95.5
90.2            95.6            96.7
92.7            93.9            98.5
91.9            96.1            98.2
88.4            93.9            97.2

media = 90.9    94.74           97
media delle medie = 94.21
